In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [7]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [9]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [10]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [11]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [12]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [13]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [14]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [15]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [16]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [27]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [29]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [31]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [33]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [35]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [37]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [39]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [42]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [44]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [46]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### Resolución

##### 1. 
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

- Primero vectorizamos los documentos y elegimos cinco índices al azar con la función random choice de numpy.  

In [100]:
import numpy as np

# 1. Vectorización de documentos 
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# número total de documentos
n_docs = X_train.shape[0]

# elegimos 5 índices aleatorios usando una funcion de numpy
indices_aleatorios = np.random.choice(n_docs, 5, replace=False)



- Para cada uno de esos índices buscamos cada documento original y los mostramos. luego buscamos los 5 documentos  mas similares de cada uno.

In [110]:
resultados = []  # lista para almacenar resultados

for idx in indices_aleatorios:
    print(f"\nDocumento original (índice {idx}) - Categoría: {newsgroups_train.target_names[y_train[idx]]}")
    
    # Calcular similaridad con todos los documentos
    cossim = cosine_similarity(X_train[idx], X_train).flatten()
    
    # Ordenar índices por similaridad (descendente), excluyendo el propio documento
    indices_ordenados = np.argsort(cossim)[::-1]
    mostsim = indices_ordenados[1:6]  # 5 más similares
    
    print("Top 5 documentos más similares:")
    
    similares = []
    for i, sim_idx in enumerate(mostsim):
        categoria_similar = newsgroups_train.target_names[y_train[sim_idx]]
        texto_similar = newsgroups_train.data[sim_idx][:300].replace("\n", " ")  # fragmento de texto
        similaridad_valor = cossim[sim_idx]
        
        print(f"  {i+1}. Índice {sim_idx} | Categoría: {categoria_similar} | Similaridad: {similaridad_valor:.3f}")
        print(f"      Texto: {texto_similar}...\n")
        
        similares.append({
            "indice": sim_idx,
            "categoria": categoria_similar,
            "texto": texto_similar,
            "similaridad": similaridad_valor
        })
    
    resultados.append({
        "indice_original": idx,
        "categoria_original": newsgroups_train.target_names[y_train[idx]],
        "texto_original": newsgroups_train.data[idx][:300].replace("\n", " "),
        "similares": similares
    })

# Resumen final
for resultado in resultados:
    print(f"\nResumen - Documento original (índice {resultado['indice_original']}) - Categoría: {resultado['categoria_original']}")
    print("Top 5 documentos más similares:")
    
    for i, sim in enumerate(resultado['similares']):
        print(f"  {i+1}. Índice {sim['indice']} | Categoría: {sim['categoria']} | Similaridad: {sim['similaridad']:.3f}")
        print(f"      Texto: {sim['texto']}...\n")


Documento original (índice 7114) - Categoría: comp.sys.ibm.pc.hardware
Top 5 documentos más similares:
  1. Índice 9493 | Categoría: comp.sys.ibm.pc.hardware | Similaridad: 0.291
      Texto: I was wondering if anyone out there has had the same problem I am having with my Gateway 2000 486-33DX VL-Bus system with ATI Graphics Ultra Pro LB.   When I have my computer in any resolution other than 800x600, everything is fine, but whenever I use it in 800x600 (Windows, AutoCAD, GIFs) the scree...

  2. Índice 1859 | Categoría: comp.sys.ibm.pc.hardware | Similaridad: 0.276
      Texto: Hi there.  We just bought a 486 DX2/66 Gateway system with a 2 meg ATI Ultra Pro video card.  Everything seems to work fine except for the Windows Drivers for 800x600 24 bit, and 800x600 and 1024x768 16 bit modes.  The fonts and icons start deteriorating after windows startup, and within minutes of ...

  3. Índice 5972 | Categoría: comp.os.ms-windows.misc | Similaridad: 0.269
      Texto: Preface: I am a novi

##### 2. 
Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [116]:
# Parámetros a probar
vectorizer_params = [
    {'ngram_range': (1,1), 'min_df':1, 'stop_words': None},
    {'ngram_range': (1,2), 'min_df':2, 'stop_words': 'english'},
    {'ngram_range': (1,3), 'min_df':3, 'stop_words': 'english'}
]

model_params = [
    {'model': MultinomialNB(alpha=1.0)},
    {'model': MultinomialNB(alpha=0.1)},
    {'model': ComplementNB(alpha=1.0)},
    {'model': ComplementNB(alpha=0.1)},
]

# Guardar resultados
resultados = []

for vect_params in vectorizer_params:
    # instanciar vectorizador con esos parámetros
    tfidfvect = TfidfVectorizer(**vect_params)
    
    # vectorizar train y test
    X_train = tfidfvect.fit_transform(newsgroups_train.data)
    X_test = tfidfvect.transform(newsgroups_test.data)
    
    for mp in model_params:
        model = mp['model']
        
        # entrenar
        model.fit(X_train, y_train)
        
        # predecir
        y_pred = model.predict(X_test)
        
        # calcular f1 macro
        f1 = f1_score(y_test, y_pred, average='macro')
        
        # guardar resultados
        resultados.append({
            'vectorizer_params': vect_params,
            'model': type(model).__name__,
            'alpha': model.alpha,
            'f1_macro': f1
        })
        
        print(f"Modelo: {type(model).__name__}, alpha={model.alpha}, vect_params={vect_params}")
        print(f"  F1-score macro: {f1:.4f}")

# Mostrar mejores resultados
best_result = max(resultados, key=lambda x: x['f1_macro'])
print("\n=== Mejor configuración ===")
print(f"Modelo: {best_result['model']}")
print(f"Alpha: {best_result['alpha']}")
print(f"Vectorizer params: {best_result['vectorizer_params']}")
print(f"F1-score macro: {best_result['f1_macro']:.4f}")

Modelo: MultinomialNB, alpha=1.0, vect_params={'ngram_range': (1, 1), 'min_df': 1, 'stop_words': None}
  F1-score macro: 0.5854
Modelo: MultinomialNB, alpha=0.1, vect_params={'ngram_range': (1, 1), 'min_df': 1, 'stop_words': None}
  F1-score macro: 0.6565
Modelo: ComplementNB, alpha=1.0, vect_params={'ngram_range': (1, 1), 'min_df': 1, 'stop_words': None}
  F1-score macro: 0.6930
Modelo: ComplementNB, alpha=0.1, vect_params={'ngram_range': (1, 1), 'min_df': 1, 'stop_words': None}
  F1-score macro: 0.6954
Modelo: MultinomialNB, alpha=1.0, vect_params={'ngram_range': (1, 2), 'min_df': 2, 'stop_words': 'english'}
  F1-score macro: 0.6504
Modelo: MultinomialNB, alpha=0.1, vect_params={'ngram_range': (1, 2), 'min_df': 2, 'stop_words': 'english'}
  F1-score macro: 0.6822
Modelo: ComplementNB, alpha=1.0, vect_params={'ngram_range': (1, 2), 'min_df': 2, 'stop_words': 'english'}
  F1-score macro: 0.7037
Modelo: ComplementNB, alpha=0.1, vect_params={'ngram_range': (1, 2), 'min_df': 2, 'stop_word

#### 3.
Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

- Transponer la matriz X_train

In [153]:
X_words = X_train.T  # transponemos: ahora filas son palabras


- Obtener el vocabulario (mapa de términos a índices):

In [158]:
vocabulario = tfidfvect.get_feature_names_out()

- Elegir 5 palabras manualmente

In [172]:
palabras_elegidas = ['cincinnati', 'fax', 'chopin', 'montreal', 'ext']

- Obtenemos sus índices en el vocabulario:

In [175]:
indices_palabras = [np.where(vocabulario == palabra)[0][0] for palabra in palabras_elegidas]


✅ 'cincinnati' está en el vocabulario.
✅ 'fax' está en el vocabulario.
✅ 'chopin' está en el vocabulario.
✅ 'montreal' está en el vocabulario.
✅ 'ext' está en el vocabulario.


- Calcular similaridad de coseno entre cada palabra y todas las demás

In [177]:
for idx in indices_palabras:
    # similaridad de la palabra elegida contra todas
    cossim = cosine_similarity(X_words[idx], X_words).flatten()
    
    # índices ordenados de mayor a menor similaridad (excluimos la propia palabra)
    indices_ordenados = np.argsort(cossim)[::-1]
    mostsim = indices_ordenados[1:6]  # top 5 similares
    
    print(f"\nPalabra base: '{vocabulario[idx]}'")
    print("Palabras más similares:")
    for i, sim_idx in enumerate(mostsim):
        print(f"  {i+1}. '{vocabulario[sim_idx]}' (similaridad: {cossim[sim_idx]:.4f})")


Palabra base: 'cincinnati'
Palabras más similares:
  1. 'booked' (similaridad: 0.4923)
  2. 'rightfully' (similaridad: 0.4279)
  3. 'hot spots' (similaridad: 0.3839)
  4. 'cyclones' (similaridad: 0.3691)
  5. '17 17' (similaridad: 0.3647)

Palabra base: 'fax'
Palabras más similares:
  1. 'fax modem' (similaridad: 0.3684)
  2. 'processing software' (similaridad: 0.3042)
  3. 'ascii format' (similaridad: 0.2979)
  4. 'service mail' (similaridad: 0.2972)
  5. 'fax data' (similaridad: 0.2940)

Palabra base: 'chopin'
Palabras más similares:
  1. '02106 chopin' (similaridad: 1.0000)
  2. 'chopin' (similaridad: 1.0000)
  3. 'chopin udel' (similaridad: 1.0000)
  4. 'chopin udel edu' (similaridad: 1.0000)
  5. '02106' (similaridad: 0.9108)

Palabra base: 'montreal'
Palabras más similares:
  1. 'pittsburgh 11' (similaridad: 0.3883)
  2. 'quebec' (similaridad: 0.3814)
  3. 'know thinking' (similaridad: 0.3727)
  4. 'damphousse' (similaridad: 0.3727)
  5. 'montreal canadiens' (similaridad: 0.3625